In [1]:
#CARREGA A(S) BIBLIOTECAS
import pandas as pd

In [7]:
#CARREGA O ARQUIVO QUE CONTEM A LOCALIZAÇÃO DAS PLANTAS DE PRODUCAO DE ÓLEO LUBRIFICANTE EM SOLO BRASILEIRO
plantas_producao = pd.read_excel("../data/boletim_1/Anexo_E.xlsx", sheetname="30-Mapa_Planta_Prod_OLAC")
plantas_producao.head()

,CNPJ,Produtor,Município,UF
0,57941890000153,AGECOM PRODUTOS DE PETRÓLEO LTDA.,MAUA,SP
1,3439153000128,BIOLUB QUÍMICA LTDA -ME,SOROCABA,SP
2,33194978000190,CASTROL BRASIL LTDA,RIO DE JANEIRO,RJ
3,45036670000104,CHEMLUB PRODUTOS QUÍMICOS LTDA,VALINHOS,SP
4,5524572001084,CHEVRON BRASIL LUBRIFICANTES LTDA.,DUQUE DE CAXIAS,RJ
5,7129683000120,CONDAT LUBRIFICANTES DO BRASIL INDÚSTRIA E COM...,BOM JESUS DOS PERDOES,SP
6,33000092003850,COSAN LUBRIFICANTES E ESPECIALIDADES S.A.,RIO DE JANEIRO,RJ
7,18235762000132,DAIDO QUÍMICA DO BRASIL LTDA,CONTAGEM,MG
8,68919158000101,DNC INDUSTRIAL CAMPINAS LTDA,HORTOLANDIA,SP
9,5092901000921,DUNAX LUBRIFICANTES LTDA. - ME,SAO GONCALO DOS CAMPOS,BA


In [9]:
#CRIA UM DICIONÁRIO PARA ARMAZENAR O UF DAS PLANTAS DE PRODUCAO
empresa_prod_uf = {}

for index,row in plantas_producao.iterrows():
    empresa = row['Produtor']
    if empresa not in empresa_prod_uf:
        empresa_prod_uf[empresa] = row['UF']
    else:
    #CASO UMA EMPRESA TENHA MAIS DE UMA PLANTA INDUSTRIAL, IMPRIMIR O NOME DA EMPRESA E A UF ONDE A PLANTA ESTÁ.    
        print(empresa+" - "+row['UF']) # A IPIERANGA NESSE CASO POSSUI DUAS PLANTAS NO MESMO ESTADO (RJ)
        continue

IPIRANGA PRODUTOS DE PETRÓLEO S.A - RJ


In [12]:
#Vendas nacionais de OLAC pelos Produtores e Importadores
#A COLUNA PRODUTOR-IMPORTADOR POSSUI A INFORMAÇÃÕ DE UMA EMPRESA QUE DECLARA UMA OPERAÇÃO DE COMERCIALIZAÇÃO À ANP.
#TODAVIA, ESSE PRODUTO COMERCIALIZADO PODE TER A SEGUINTE ORIGEM:
### 1. SER PRODUZIDO EM PLANTA INDUSTRIAL PRÓPRIA EM SOLO BRASILEIRO.
### 2. SER PRODUZIDO EM PLANTA INDUSTRIAL DE TERCEIRO EM SOLO BRASILEIRO.
### 3. SER PRODUZID FORA DO BRASIL (PLANTA PRÓPRIA OU NÃO.)
vendas = pd.read_excel("../data/boletim_1/Anexo_H.xlsx", sheetname="41-Vendas OLAC Geral")
vendas.head(5)

,Data,Cód Regulado ANP,Produtor-Importador,Código do Produto,Descrição do Produto,Região do Destinatário,UF do Destinatário,Volume-Litros
0,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,SUL,PR,9828
1,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620101002,ENGRENAGENS E SISTEMAS CIRCULATÓRIOS,SUL,SC,800
2,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620501001,Ciclo Otto,SUL,PR,3685
3,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620501001,Ciclo Otto,SUL,RS,480
4,12/2016,200296,TG Indústria e Comércio de Lubrificantes e Gra...,620501001,Ciclo Otto,SUL,SC,5040


In [13]:
#TROCAR O NOME DA EMPRESA (PRODUTOR-IMPORTADOR) PELO UF ONDE ESTÁ LOCALIZADO A PLANTA DE PRODUÇÃO.
vendas['Produtor-Importador'] = vendas['Produtor-Importador'].map(empresa_prod_uf)

In [15]:
#NÚMERO DE REGISTROS DE DECLARAÇÃO DE COMERCIALIZAÇÃO DE EMPRESAS QUE NÃO PRODUZIRAM EM PLANTA PRÓPRIA NO BRASIL OU SÃO 
#CONSIDERADAS IMPORTADORAS
vendas['Produtor-Importador'].isnull().sum()

9848

In [27]:
#PERCENTUAL DE REGISTROS DE DECLARAÇÃO DE COMERCIALIZAÇÃO QUE SÃO DE EMPRESAS QUE NÃO PRODUZIRAM EM PLANTA PRÓPRIA NO BRASIL OU SÃO 
#CONSIDERADAS IMPORTADORAS
str(round(vendas['Produtor-Importador'].isnull().sum()/vendas['Produtor-Importador'].shape[0]*100,2))+"%"

'26.14%'

In [31]:
#SOMA DOS VOLUMES DE COMERCIALIZACAO DE EMPRESAS DETENTORAS & PRODUTORAS
count_volume_detentor_produtor = 0

#SOMA DOS VOLUMES DE EMPRESAS QUE SÃO DETENTORAS/TERCERIZADORAS OU DETENTORAS/IMPORTADORAS
count_volume_outros = 0
for index,row in vendas.iterrows():
    if type(row['Produtor-Importador']) != str:
        count_volume_outros += row['Volume-Litros']
    else:
        count_volume_detentor_produtor += row['Volume-Litros']

In [47]:
print("Volume total de empresas Detentoras/Produtoras "+str(count_volume_detentor_produtor/1000)+ " m³")
print("Volume total de empresas Detentoras/Tercerizadoras/Importadoras "+str(count_volume_outros/1000)+ " m³")

Volume total de empresas Detentoras/Produtoras 1131775.986 m³
Volume total de empresas Detentoras/Tercerizadoras/Importadoras 97916.397 m³


In [52]:
#PERCENTUAL EM VOLUME DOS REGISTROS QUE NÃO PODE SE INFERIR QUAL A ORIGEM DA PRODUÇÃO, APENAS O DESTINATÁRIO.
str(round(count_volume_outros/(count_volume_outros+count_volume_detentor_produtor)*100,2))+"%"

'7.96%'